In [1]:
#Import libraries:
import cPickle
import csv
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold

In [2]:
X = np.empty((0,48), float)
Y = np.empty(0, str)
i = 0
r = csv.reader(open('Features/features.csv',"rb"),delimiter=',') # Here your csv file
for l in r:
    if i == 1:
        if not np.isnan(map(float,l)).any():
            X = np.append(X, np.array([l[0:48]]), axis=0)
            Y = np.append(Y, np.array([l[48]]), axis=0)
    else:
        i = 1
        features = l[0:48]
        output = l[48]
        
X = X.astype(float)
Y = Y.astype(str)

In [11]:
# fit model on training data
model = xgb.XGBClassifier(learning_rate =0.1,
 n_estimators=500,
 max_depth=3,
 objective= 'multi:softmax',
 nthread=4,
 seed=27)

In [12]:
error=[]
kf = KFold(n_splits=10,shuffle = True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model.fit(X_train,Y_train)
    test_set_predictions = model.predict(X_test)

    test_misclassification_percentage = 0.0
    for i in range(len(test_set_predictions)):
        if test_set_predictions[i]!=Y_test[i]:
            test_misclassification_percentage+=1.0
    test_misclassification_percentage *= 100.0/len(Y_test)
    error.append(test_misclassification_percentage)
print 'accuracy =', (100.0-np.mean(error)), '%'

accuracy = 79.2776447703 %


In [14]:
model = xgb.XGBClassifier(max_depth=3, n_estimators=500, learning_rate=0.1,nthread=4,seed=27 ,objective='multi:softmax').fit(X,Y)
with open('model.pkl', 'wb') as fid:
    cPickle.dump(model, fid)